In [ ]:
# import dependencies
import json
import openrouteservice
import os
from oauth2client.service_account import ServiceAccountCredentials
import gspread


In [ ]:
# manually set lat/lng here
locations = {

}

In [ ]:
# authenticate and connect to google sheets
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]
creds = ServiceAccountCredentials.from_json_keyfile_name("api_keys.json", scope)
client = gspread.authorize(creds)
spreadsheet = client.open_by_key("12L4EkdRqaQ_e42fGHWaTmgCeqQrNgjTfoeAEc5AB6tw")

In [ ]:
# load route data
route_sheet = spreadsheet.worksheet("Routes")
route_data = pd.DataFrame(route_sheet.get_all_records())

In [13]:
# connect to OpenRouteService API for automobile routes
with open("api_keys.json") as f:
    api_keys = json.load(f)
ors_client = openrouteservice.Client(key=api_keys["openrouteservice"])

In [14]:
# function to return a geojson route between two coordinates 
def fetch_route(start_coords, end_coords, transport_mode="driving-car"):
    try:
        # Fetch the route from OpenRouteService
        route = ors_client.directions(
            coordinates=[start_coords, end_coords],
            profile=transport_mode,
            format="geojson"
        )
        return route
    except openrouteservice.exceptions.ApiError as e:
        print(f"Failed to fetch route: {e}")
        return None
    
# function to save the route to a geojson file
def save_route_as_geojson(route, start_location, end_location):
    filename = f"{start_location.replace(' ', '_')}_to_{end_location.replace(' ', '_')}.geojson"
    filepath = os.path.join("../docs/resources/geojson", filename)
    with open(filepath, "w") as f:
        json.dump(route, f, indent=4)
    print(f"Route saved to {filepath}")

Failed to fetch route: 404 ({'error': {'code': 2010, 'message': 'Could not find routable point within a radius of 350.0 meters of specified coordinate 0: -19.1817000 15.9177000.; Could not find routable point within a radius of 350.0 meters of specified coordinate 1: -22.5609000 17.0658000.'}, 'info': {'engine': {'build_date': '2025-03-14T11:07:03Z', 'graph_version': '1', 'version': '9.1.1'}, 'timestamp': 1743837194919}})
